# [Soy Bean Desease (Small) Dataset](https://archive.ics.uci.edu/dataset/91/soybean+small)

A small subset of Michalski's famous soybean disease database

### Estrutura do dataset

| Alvo  | Valores |
| ------------- | ------------- |
|class| {D1, D2, D3, D4, }| 

| Features  | Valores |
| ------------- | ------------- |
|date| {4, 5, 3, 6, 0, 2, 1, }| 
|plant-stand| {0, 1, }| 
|precip| {2, 0, 1, }| 
|temp| {1, 2, 0, }| 
|hail| {1, 0, }| 
|crop-hist| {1, 3, 2, 0, }| 
|area-damaged| {0, 1, 2, 3, }| 
|severity| {1, 2, }| 
|seed-tmt| {0, 1, }| 
|germination| {2, 1, 0, }| 
|plant-growth| {1, }| 
|leaves| {1, 0, }| 
|leafspots-halo| {0, }| 
|leafspots-marg| {2, }| 
|leafspot-size| {2, }| 
|leaf-shread| {0, }| 
|leaf-malf| {0, }| 
|leaf-mild| {0, }| 
|stem| {1, }| 
|lodging| {0, 1, }| 
|stem-cankers| {3, 0, 1, 2, }| 
|canker-lesion| {1, 0, 3, 2, }| 
|fruiting-bodies| {1, 0, }| 
|external-decay| {1, 0, }| 
|mycelium| {0, 1, }| 
|int-discolor| {0, 2, }| 
|sclerotia| {0, 1, }| 
|fruit-pods| {0, 3, }| 
|fruit-spots| {4, }| 
|seed| {0, }| 
|mold-growth| {0, }| 
|seed-discolor| {0, }| 
|seed-size| {0, }| 
|shriveling| {0, }| 
|roots| {0, 1, }| 

### Instalação das dependências

In [13]:
# !pip install ucimlrepo
# !pip install tensorflow
# !pip install sklearn
# !pip install pandas

In [14]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Dense

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

from ucimlrepo import fetch_ucirepo 
pd.options.mode.copy_on_write = True

### Importando o dataset

In [15]:
# fetch dataset 
dataset = fetch_ucirepo(id=91) 
  
# data (as pandas dataframes) 
X = dataset.data.features 
Y = dataset.data.targets 

# Transform labels to int
labels = Y["class"].unique()
for i in range(len(labels)):
  Y.loc[Y['class']==labels[i], 'class'] = i


### One-hot Encoding das features

In [16]:
cat_fatures = []
for feature in dataset.data.features:
    #print(feature)
    feature_info = dataset.variables.loc[dataset.variables["name"] == feature]
    if feature_info.type.values[0] == "Categorical":
        cat_fatures.append(feature)


In [17]:
X = pd.get_dummies(X,columns=cat_fatures, dtype=int)

### Dividindo os conjuntos de teste e treino

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size= 0.3, random_state = 28)

### One-Hot Enconding dos Targets

In [19]:
y_train = to_categorical(y_train) 
y_true = list(y_test['class'])
y_test = to_categorical(y_test) 

### Perceptron

In [20]:
model = tf.keras.Sequential([
    Flatten(input_shape=(len(X.columns),)),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(len(y_train[0]), activation='softmax')
])

In [21]:
model.compile(
  loss='categorical_crossentropy', 
  optimizer='sgd', 
  metrics=['Accuracy', 'Precision', 'Recall', 'F1Score']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 72)                0         
                                                                 
 dense_3 (Dense)             (None, 64)                4672      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 4)                 516       
                                                                 
Total params: 13508 (52.77 KB)
Trainable params: 13508 (52.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Treinamento

In [22]:
model.fit(x_train, y_train, epochs=300, batch_size=32, verbose=1)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/300
1/1 [==============================] - 1s 532ms/step - loss: 1.4434 - Accuracy: 0.2188 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.1623
Epoch 2/300
1/1 [==============================] - 0s 5ms/step - loss: 1.4218 - Accuracy: 0.2188 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.1623
Epoch 3/300
1/1 [==============================] - 0s 5ms/step - loss: 1.4014 - Accuracy: 0.2500 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.1985
Epoch 4/300
1/1 [==============================] - 0s 6ms/step - loss: 1.3820 - Accuracy: 0.2500 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.1985
Epoch 5/300
1/1 [==============================] - 0s 5ms/step - loss: 1.3633 - Accuracy: 0.3125 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.2473
Epoch 6/300
1/1 [==============================] - 0s 6ms/step - loss: 1.3454 - Accuracy: 0.3438 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.2885
Epoch 7/300
1/1 [===============

[0.17068937420845032, 1.0, 1.0, 1.0, array([1., 1., 1., 1.], dtype=float32)]

In [23]:
predictions = model.predict(x_test)
predictions = [list(p).index(max(p)) for p in predictions]

1/1 [==============================] - 0s 44ms/step


In [24]:
print(f'Acurácia obtida: {accuracy_score(y_true, predictions) * 100:.2f}%')
print(f'Precisão obtida: {precision_score(y_true, predictions, average="macro") * 100:.2f}%')
print(f'Recall obtido: {recall_score(y_true, predictions, average="macro") * 100:.2f}%')
print(f'F1 Score obtida: {f1_score(y_true, predictions, average="macro") * 100:.2f}%')


Acurácia obtida: 100.00%
Precisão obtida: 100.00%
Recall obtido: 100.00%
F1 Score obtida: 100.00%
